In [14]:
import os
from dotenv import load_dotenv

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# print(PINECONE_API_KEY)

In [15]:
%pwd

'd:\\Ashok\\AI\\CURSOR\\__MEDIBLAZE\\src'

In [16]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [17]:
def load_pdf_file(data_dir="."):
    # Resolve path relative to current working directory or file
    data_path = os.path.abspath(data_dir)
    loader = DirectoryLoader(data_path, glob="**/*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [18]:
extracted_data = load_pdf_file("../Data")

In [33]:
#CHunking
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [34]:
print(len(text_split(extracted_data)))

5859


In [35]:
from langchain_pinecone import PineconeEmbeddings
# Initialize Pinecone embeddings model
embeddings = PineconeEmbeddings(model="llama-text-embed-v2")
# embeddings = PineconeEmbeddings(model="multilingual-e5-large")

In [36]:
text_chunks = text_split(extracted_data)

In [41]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "mediblaze-bot"
if index_name in pc.list_indexes():
    pc.delete_index(index_name)


In [51]:
import requests

url = "https://controller.us-east-1.pinecone.io/databases"

headers = {
    "Api-Key":PINECONE_API_KEY,
    "Content-Type": "application/json"
}

payload = {
    "name": index_name,
    "cloud": "aws",
    "region": "us-east-1",
    "model": {
        "name": "llama-text-embed-v2",
        "input_type": "text",
        "field_map": {
            "text": "text"
        }
    }
}

response = requests.post(url, json=payload, headers=headers)

print(response.status_code, response.text)


docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
    batch_size=50 
)
print(f"Successfully uploaded {len(text_chunks)} chunks to Pinecone index")


ConnectionError: HTTPSConnectionPool(host='controller.us-east-1.pinecone.io', port=443): Max retries exceeded with url: /databases (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000279AF84CCD0>: Failed to resolve 'controller.us-east-1.pinecone.io' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec
# load_dotenv()

# pc = Pinecone(api_key=PINECONE_API_KEY)
# index_name = "dhurba-furniture-rag"

# if index_name not in pc.list_indexes():
#     print(f"Creating index {index_name}...")
#     pc.create_index(
#         name=index_name,
#         dimension=384,  # Dimension of the embeddings
#         metric="cosine",  # Similarity metric
#         spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#         )
#     )


In [ ]:
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name = index_name,
#     embedding=embeddings,
# )

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
retrieved_docs = retriever.invoke("What types of furniture does Dhurba Furniture specialize in")

In [ ]:
# retrieved_docs

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore
# from pinecone.grpc import PineconeGRPC as Pinecone
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    api_key = os.getenv("GOOGLE_API_KEY"),
    temperature=0.2,
    max_output_tokens=100,
)


FROM ExTSINF INDEX

In [ ]:

# Connect to your Pinecone index
index_name = "mediblaze" # 👈 Replace with your index name
vectorstore = PineconeVectorStore(index_name=index_name,
                                  embedding=embeddings, 

                                  pinecone_api_key=PINECONE_API_KEY)

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '0d54e4bf86582f4e28bd8ef3dfbfa17d', 'date': 'Thu, 19 Jun 2025 19:14:09 GMT', 'server': 'Google Frontend', 'Content-Length': '95', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource dhurba-furniture-rag not found"},"status":404}


In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "search_document_dataset", 
    "searches and returns relevant information from the document database (furniture.pdf)."
)

query = "What payment methods do you accept?"
response = retriever_tool.invoke(query)
print("\nResponse:")
print(response)


Response:
●  Custom  Manufacturing:  Production  of  bespoke  furniture  pieces.  ●  Delivery  &  Installation:  Safe  transport  and  professional  assembly  of  
furniture.
 ●  After-Sales  Support:  Assistance  with  product  inquiries,  maintenance  tips,  
and
 
warranty
 
claims.
 ●  Repair  Services:  (Potentially)  Offering  repair  services  for  furniture  purchased  
from
 
them,
 
subject
 
to
 
terms.
 Payment  Policy:  
●  Deposit:  A  non-refundable  deposit  (e.g.,  50%)  is  typically  required  for  custom  
orders
 
to
 
initiate
 
production.
 ●  Balance  Payment:  The  remaining  balance  is  due  upon  completion  of  the  
furniture,
 
prior
 
to
 
delivery
 
or
 
pickup.
 ●  Payment  Methods:  Acceptance  of  various  methods  like  bank  transfer,  
credit/debit
 
cards.
 Delivery  Policy:  
●  Scheduling:  Delivery  dates  are  scheduled  in  advance  with  the  customer.  ●  Access:  Customers  are  responsible  for  ensuring  clear  access  to  the  deliver

In [ ]:
from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "search_document_dataset",
    "searches and returns relevant information from the document database (furniture.pdf)."
)

query = "Whatpaymentmethodsdoyouaccept?"
retriever_tool.invoke(query)


'●  Custom  Manufacturing:  Production  of  bespoke  furniture  pieces.  ●  Delivery  &  Installation:  Safe  transport  and  professional  assembly  of  \nfurniture.\n ●  After-Sales  Support:  Assistance  with  product  inquiries,  maintenance  tips,  \nand\n \nwarranty\n \nclaims.\n ●  Repair  Services:  (Potentially)  Offering  repair  services  for  furniture  purchased  \nfrom\n \nthem,\n \nsubject\n \nto\n \nterms.\n Payment  Policy:  \n●  Deposit:  A  non-refundable  deposit  (e.g.,  50%)  is  typically  required  for  custom  \norders\n \nto\n \ninitiate\n \nproduction.\n ●  Balance  Payment:  The  remaining  balance  is  due  upon  completion  of  the  \nfurniture,\n \nprior\n \nto\n \ndelivery\n \nor\n \npickup.\n ●  Payment  Methods:  Acceptance  of  various  methods  like  bank  transfer,  \ncredit/debit\n \ncards.\n Delivery  Policy:  \n●  Scheduling:  Delivery  dates  are  scheduled  in  advance  with  the  customer.  ●  Access:  Customers  are  responsible  for  ensurin

In [ ]:
from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "search_document_dataset",
    "searches and returns relevant information from the document database (furniture.pdf)."
)

tools = [retriever_tool]

In [ ]:
# Let's test if the retriever tool works independently first
test_query = "What are the delivery options?"
tool_response = retriever_tool.invoke(test_query)
print("Direct tool response:")
print(tool_response)
print("\n" + "="*50 + "\n")

# Let's also test with a more direct query about delivery
delivery_query = "delivery policy"
delivery_response = retriever_tool.invoke(delivery_query)
print("Delivery policy response:")
print(delivery_response)

Direct tool response:
from
 
4
 
to
 
12
 
weeks.
 
We
 
will
 
provide
 
an
 
estimated
 
delivery
 
date
 
upon
 
order
 
confirmation.
 Q5:  What  payment  methods  do  you  accept?  A5:  We  typically  accept  various  payment  methods  including  bank  transfers,  major  
credit/debit
 
cards,
 
and
 
sometimes
 
installment
 
plans
 
for
 
larger
 
orders.
 
A
 
deposit
 
is
 
usually
 
required
 
to
 
confirm
 
custom
 
orders.
 Q6:  Do  you  offer  delivery  and  installation  services?  A6:  Yes,  we  provide  professional  delivery  and  installation  services  for  all  purchased  
furniture.
 
Our
 
team
 
ensures
 
your
 
furniture
 
is
 
delivered
 
safely
 
and
 
assembled
 
correctly
 
in
 
your
 
desired
 
location.
 Q7:  What  is  your  warranty  policy?  A7:  We  stand  by  the  quality  of  our  products.  Our  furniture  typically  comes  with  a  warranty  
covering
 
manufacturing
 
defects.
 
The
 
duration
 
and
 
terms
 
of
 
the
 
warranty
 
may
 
vary
 
by
 

In [ ]:
from langgraph.prebuilt import create_react_agent

from langchain_core.messages import HumanMessage

agent_executor = create_react_agent(llm, tools)

# Test the agent with a simple query
response = agent_executor.invoke({"messages": [HumanMessage(content="What are your delivery options?")]})

print("--- Agent's Response ---")
print(response["messages"][-1].content)

--- Agent's Response ---
I am sorry, I cannot answer your question. I do not have the ability to look up delivery options.


In [ ]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm, tools)

from langchain_core.messages import HumanMessage
# --- 6. Example Usage of the Agent ---

print("--- Agent ready. Asking a question ---")

# The agent will decide whether to use the tool based on the query
# For queries about Dhurba Furniture details, it should use the tool
response = agent_executor.invoke({"messages": [HumanMessage(content="What payment methods do you accept")]})

print("\n--- Agent's Response ---")
print(response["messages"][-1].content)

print("\n--- Asking another question (agent might use the tool again) ---")
response_customization = agent_executor.invoke({"messages": [HumanMessage(content="What customization options are available for materials like wood and upholstery fabrics?")]})
print("\n--- Agent's Response on Customization ---")
print(response_customization["messages"][-1].content)

--- Agent ready. Asking a question ---

--- Agent's Response ---
I am sorry, but I am unable to answer the question about payment methods. I can only search for information within the document database.

--- Asking another question (agent might use the tool again) ---

--- Agent's Response on Customization ---
For wood, you can choose from various hardwoods like teak, oak, walnut, rosewood, and mahogany. For upholstery fabrics, you can select from a vast array of fabrics such as linen, velvet, leather, and synthetic blends in different colors, patterns, and textures.
